In [2]:
# import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

import pandas as pd
import numpy as np
import datetime as dt

import sqlite3
import sqlalchemy as db
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

from flask import Flask, jsonify, render_template, request

import warnings
warnings.filterwarnings('ignore')

In [3]:
# sqlite3 nift
con = sqlite3.connect("movie_ratings_db.sqlite")
links_df = pd.read_sql_query("SELECT * FROM Links", con)
movies_df = pd.read_sql_query("SELECT * FROM Movies", con)
ratings_df = pd.read_sql_query("SELECT * FROM Ratings", con)
tags_df = pd.read_sql_query("SELECT * FROM Tags", con)

In [4]:
# i want retain all the associated genre's per movie
# lets try a get_dummies split, and then concatenate as a categorical
genres_split = movies_df['genres'].str.get_dummies('|')
movies_encoded = pd.concat([movies_df, genres_split], axis=1)
# drop original genres column
movies_encoded.drop('genres', axis=1, inplace=True)

In [5]:
# prep tags_df, removing timestamp, unsure why it would be indicative to include when they selected a tag
tags_df = tags_df.drop(columns='timestamp')
ratings_df = ratings_df.drop(columns='timestamp')

In [6]:
# merging tags with movies_encoded, left to retain all tag rows as that's our bulk
tags_movies_df = pd.merge(tags_df, movies_encoded, on='movieId', how='left')
# i forgot about ratings, so here it is
tags_movies_ratings_df = pd.merge(tags_movies_df, ratings_df, on='userId', how='left')
# forgot a bunch of other stuff
tags_movies_ratings_df_revised = tags_movies_ratings_df.drop(columns=['movieId_x','movieId_y'])

In [16]:
#*
# rewrite for ratings only
tags_ratings_df = pd.merge(tags_df, ratings_df, on='movieId', how='left')
tags_ratings_df_rev = tags_ratings_df.drop(columns=['userId_y'])
tags_ratings_df_rev = pd.get_dummies(tags_ratings_df_rev, dtype=float)

In [ ]:
# awful, coll filter
y = tags_ratings_df_rev['userId']
X = tags_ratings_df_rev.drop(columns=['userId'])

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

nn = tf.keras.models.Sequential()

ValueError: could not convert string to float: 'racism'

In [ ]:
nn.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
# 0.4 indicates that 40% of the input units will be deactivated, resetting after each cycle
nn.add(tf.keras.layers.Dropout(0.4))
# layer 2, dropout itself is not a hidden layer
nn.add(tf.keras.layers.Dense(64, activation='relu'))
# layer 3
nn.add(tf.keras.layers.Dense(32, activation='relu'))
# layer 4, def prob function - sigmoid
nn.add(tf.keras.layers.Dense(1, activation='sigmoid'))

nn.summary()

nightmare nightmare nightmare

In [7]:
# let's bin; this is a nightmare; no bins until my computer crashes
# my computer crashed
tags_movies_ratings_df_revised.columns

Index(['userId', 'tag', 'title', '(no genres listed)', 'Action', 'Adventure',
       'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'rating'],
      dtype='object')

In [8]:
# collaborative filtering, we wanna pair users of like with other users of like
# so in this instance our target is 'userId'
y = tags_movies_ratings_df_revised['userId']
# X is everything else BUT we need to preprocess for categorical and numerical and run them separately
X = tags_movies_ratings_df_revised.drop(columns = 'userId')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# separating numerical and categorical columns, unsure if rating need be categorical?
numerical_columns = ['(no genres listed)', 'Action', 'Adventure',
                    'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                    'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical',
                    'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
                    'rating']
categorical_columns = ['tag','title']

# scaler for numerical portion
scaler = StandardScaler()
X_train_numerical = scaler.fit_transform(X_train[numerical_columns])
X_test_numerical = scaler.transform(X_test[numerical_columns])

# dummy var creation for categorical
X_train_categorical = pd.get_dummies(X_train[categorical_columns])
X_test_categorical = pd.get_dummies(X_test[categorical_columns])

In [18]:
# numerical shape input layer
numerical_input = tf.keras.layers.Input(shape=(len(numerical_columns),))

# create embedding for categorical layers
embeddings = []
for col in categorical_columns:
    input_dim = len(tags_movies_ratings_df_revised[col].unique())
    embedding_dim = min(50, (input_dim + 1) // 2)
    input_layer = tf.keras.layers.Input(shape=(1,))
    embedding = tf.keras.layers.Embedding(input_dim=input_dim, output_dim=embedding_dim)(input_layer)
    embedding = tf.keras.layers.Flatten()(embedding)
    embeddings.append(embedding)

In [1]:
# concat numerical and embedded categorical
concatenated = tf.keras.layers.concatenate([numerical_input]) # +embedding

# activations for hidden layers, output of previous fed into following
dense_layer_1 = tf.keras.layers.Dense(64, activation='relu')(concatenated)
dense_layer_2 = tf.keras.layers.Dense(32, activation='selu')(dense_layer_1)
# i don't think sigmoid is going to work here
final_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dense_layer_2)

NameError: name 'tf' is not defined